# Built-in Middlewares for HR Agents - LangChain 1.0 (Updated)

**Module:** Built-in Middleware Components - Class-Based Approach

**What you'll learn:**
- 📝 SummarizationMiddleware - Automatic conversation summarization using class-based middleware
- 👤 HumanInTheLoopMiddleware - Manual approval workflows with built-in middleware
- 🎯 Production patterns for HR use cases with LangChain 1.0

**Key Updates in This Version:**
- ✅ Uses built-in `SummarizationMiddleware` and `HumanInTheLoopMiddleware` classes
- ✅ Correct parameter: `interrupt_on` (not `tool_configs`)
- ✅ Correct parameter: `system_prompt` (not `prompt`)
- ✅ No more manual hook implementations
- ✅ Cleaner, more maintainable code

**HR Use Cases:**
- Long employee consultation sessions with memory management
- Critical HR decisions requiring manager approval
- Salary updates with multi-level authorization
- Compliance-driven approval workflows

**Time:** 1-2 hours

---

## Setup: Install Dependencies

In [ ]:
# Install LangChain 1.0 and middleware packages
!pip install --pre -U langchain langchain-openai langgraph
!pip install langgraph-checkpoint-sqlite

## Setup: Configuration and Imports

In [ ]:
# Configure API key
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Imports - Updated for LangChain 1.0
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware, HumanInTheLoopMiddleware
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from typing import Annotated
from datetime import datetime
import json

print("✅ Setup complete with LangChain 1.0 middleware!")

## Setup: HR Tools and Data

In [ ]:
# HR Employee Database
EMPLOYEES = {
    "101": {
        "name": "Priya Sharma",
        "department": "Engineering",
        "role": "Senior Developer",
        "salary": 120000,
        "leave_balance": 12,
        "manager_id": "102"
    },
    "102": {
        "name": "Rahul Verma",
        "department": "Engineering",
        "role": "Engineering Manager",
        "salary": 180000,
        "leave_balance": 8,
        "manager_id": "103"
    },
    "103": {
        "name": "Anjali Patel",
        "department": "HR",
        "role": "HR Director",
        "salary": 200000,
        "leave_balance": 15,
        "manager_id": None
    },
    "104": {
        "name": "Arjun Reddy",
        "department": "Sales",
        "role": "Sales Team Lead",
        "salary": 150000,
        "leave_balance": 10,
        "manager_id": "105"
    },
    "105": {
        "name": "Sneha Gupta",
        "department": "Sales",
        "role": "Sales Director",
        "salary": 190000,
        "leave_balance": 5,
        "manager_id": "103"
    }
}

# Define HR tools
@tool
def get_employee_info(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Get employee information by ID."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"{emp['name']} - {emp['department']} - {emp['role']}"
    return f"Employee {employee_id} not found"

@tool
def check_leave_balance(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Check leave balance for an employee."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"{emp['name']} has {emp['leave_balance']} days of leave remaining"
    return f"Employee {employee_id} not found"

@tool
def get_salary_info(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Get salary information. SENSITIVE operation."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"{emp['name']}'s annual salary: ₹{emp['salary']:,}"
    return f"Employee {employee_id} not found"

@tool
def update_salary(
    employee_id: Annotated[str, "Employee ID"],
    new_salary: Annotated[int, "New salary amount"]
) -> str:
    """Update employee salary. CRITICAL operation requiring approval."""
    if employee_id in EMPLOYEES:
        old_salary = EMPLOYEES[employee_id]['salary']
        EMPLOYEES[employee_id]['salary'] = new_salary
        return f"✅ Salary updated for {EMPLOYEES[employee_id]['name']}: ₹{old_salary:,} → ₹{new_salary:,}"
    return f"Employee {employee_id} not found"

@tool
def approve_leave(
    employee_id: Annotated[str, "Employee ID"],
    days: Annotated[int, "Number of leave days"]
) -> str:
    """Approve leave request. Requires manager approval."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        if emp['leave_balance'] >= days:
            EMPLOYEES[employee_id]['leave_balance'] -= days
            return f"✅ Approved {days} days leave for {emp['name']}. Remaining: {EMPLOYEES[employee_id]['leave_balance']} days"
        return f"❌ Insufficient leave balance. {emp['name']} has only {emp['leave_balance']} days"
    return f"Employee {employee_id} not found"

print("✅ HR tools and data configured!")
print(f"Total employees: {len(EMPLOYEES)}")

---
# Part 1: SummarizationMiddleware 📝

## Why Summarization?

**Problem:** Long HR consultation sessions exceed LLM context windows

**Solution:** Built-in SummarizationMiddleware automatically:
- Monitors message token count
- Summarizes old messages when threshold reached
- Keeps recent messages intact
- Preserves conversation context

---

## Lab 1.1: Basic Summarization Setup with Built-in Middleware

In [ ]:
# Create HR consultation agent with built-in SummarizationMiddleware
hr_consultation_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance, get_salary_info],
    middleware=[
        SummarizationMiddleware(
            model="openai:gpt-4o-mini",  # Model for generating summaries
            max_tokens_before_summary=500,  # Low threshold for demo
            messages_to_keep=3,  # Keep last 3 messages
            summary_prompt="Summarize the HR consultation conversation concisely, preserving key employee details and requests."
        )
    ],
    checkpointer=InMemorySaver(),
    system_prompt="""You are a helpful HR consultant.
    
    Help employees with:
    - General information
    - Leave balance inquiries
    - Career guidance
    - Policy questions
    
    Be friendly, professional, and remember conversation context."""
)

print("✅ HR Consultation Agent with SummarizationMiddleware ready!")
print("\nMiddleware Configuration:")
print("  - Model: openai:gpt-4o-mini")
print("  - Token threshold: 500")
print("  - Messages to keep: 3")

## Lab 1.2: Test Long Conversation with Auto-Summarization

In [ ]:
config = {"configurable": {"thread_id": "consultation_session_1"}}

# Simulate long consultation session
consultation_messages = [
    "Hi, I'm Priya Sharma, employee 101. I have some questions.",
    "I joined the company in 2020 and I work in the Engineering department.",
    "Can you tell me about my current role and responsibilities?",
    "I'm interested in understanding the career progression path for senior developers.",
    "What are the typical skills needed to become an Engineering Manager?",
    "How many days of leave do I have remaining this year?",
    "I'm planning to take a vacation next month. What's the leave approval process?",
    "Can you check my salary information?",
    "What benefits am I eligible for at my current level?",
    "Tell me about the company's professional development programs."
]

print("=" * 70)
print("LONG HR CONSULTATION SESSION WITH AUTO-SUMMARIZATION")
print("=" * 70)

for i, message in enumerate(consultation_messages, 1):
    print(f"\n{'='*70}")
    print(f"Turn {i}/10")
    print(f"{'='*70}")
    print(f"👤 Priya: {message}")
    
    result = hr_consultation_agent.invoke(
        {"messages": [{"role": "user", "content": message}]},
        config
    )
    
    response = result['messages'][-1].content
    print(f"\n🤖 HR Agent: {response[:200]}...")
    print(f"\nTotal messages in state: {len(result['messages'])}")
    
    # Check if summarization occurred
    if any('summary' in str(msg).lower() for msg in result['messages'][:2]):
        print("📝 Note: Summarization occurred - older messages condensed!")

print("\n" + "=" * 70)
print("✅ Long conversation handled with automatic summarization!")
print("\n💡 The middleware automatically summarized older messages")
print("   when token count exceeded 500, keeping the last 3 messages intact.")

---
# Part 2: HumanInTheLoopMiddleware 👤

## Why Human-in-the-Loop?

**Critical HR Operations Need Approval:**
- 💰 Salary updates
- 🗑️ Employee termination
- 📝 Contract changes
- 🎯 Performance reviews
- 🏆 Promotions

**Built-in HumanInTheLoopMiddleware Features:**
- Intercepts tool calls before execution
- Creates interrupts for approval
- Works seamlessly with LangGraph checkpointers
- Supports tool-level configuration
- Provides resumption after approval

---

## Lab 2.1: Understanding `interrupt_on` Parameter

The `interrupt_on` parameter configures which tools require approval:

```python
interrupt_on={
    "tool_name": True,  # All decisions allowed (approve/edit/reject)
    "another_tool": {"allowed_decisions": ["approve", "reject"]},  # No editing
    "safe_tool": False  # No approval needed
}
```

**Decision Types:**
- `approve`: Execute tool as-is
- `edit`: Modify arguments before execution
- `reject`: Cancel tool execution with feedback

## Lab 2.2: HR Operations with Built-in Approval Workflow

In [ ]:
# Create agent with built-in HumanInTheLoopMiddleware
hr_approval_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance, get_salary_info, update_salary, approve_leave],
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "update_salary": True,  # All decisions: approve/edit/reject
                "approve_leave": True   # All decisions: approve/edit/reject
            }
        )
    ],
    checkpointer=InMemorySaver(),  # Required for interrupts
    system_prompt="""You are an HR operations assistant.
    
    You help with:
    - Employee information lookup
    - Leave balance checks
    - Salary updates (requires approval)
    - Leave approvals (requires manager approval)
    
    Always be professional and follow proper procedures."""
)

print("✅ HR Agent with HumanInTheLoopMiddleware ready!")
print("\nTools requiring approval:")
print("  - update_salary (all decisions: approve/edit/reject)")
print("  - approve_leave (all decisions: approve/edit/reject)")

## Lab 2.3: Test Approval Workflow - Salary Update

In [ ]:
print("=" * 70)
print("SCENARIO: Salary Update Request with Approval")
print("=" * 70)

config = {"configurable": {"thread_id": "salary_update_1"}}

# Step 1: Request salary update
print("\n📝 Step 1: Request salary update...\n")

result = hr_approval_agent.invoke(
    {"messages": [{"role": "user", "content": "Please update the salary for employee 101 (Priya Sharma) to ₹150,000"}]},
    config
)

print(f"\nAgent Response: {result['messages'][-1].content[:300]}...")

# Step 2: Check for interrupts
print("\n" + "=" * 70)
print("⏸️  Step 2: Check for pending approvals...")
print("=" * 70)

# Check if interrupt occurred
if '__interrupt__' in result:
    print("\n✋ INTERRUPT DETECTED!")
    print("\nInterrupt Details:")
    interrupt = result['__interrupt__'][0]
    action_request = interrupt.value['action_requests'][0]
    print(f"  Tool: {action_request['name']}")
    print(f"  Arguments: {action_request['arguments']}")
    print(f"\n  Allowed decisions: {interrupt.value['review_configs'][0]['allowed_decisions']}")
else:
    print("\n💡 The agent has paused execution waiting for approval.")
    print("   In a production system, you would:")
    print("   1. Notify the approving manager")
    print("   2. Display approval UI with tool call details")
    print("   3. Resume execution after approval/rejection")

# Step 3: Show how to approve
print("\n" + "=" * 70)
print("✅ Step 3: How to approve and resume...")
print("=" * 70)

print("""
To approve and resume execution:

from langgraph.types import Command

# Approve
result = hr_approval_agent.invoke(
    Command(resume={
        "decisions": [{"type": "approve"}]
    }),
    config
)

# Edit and approve
result = hr_approval_agent.invoke(
    Command(resume={
        "decisions": [{
            "type": "edit",
            "args": {"employee_id": "101", "new_salary": 145000}
        }]
    }),
    config
)

# Reject
result = hr_approval_agent.invoke(
    Command(resume={
        "decisions": [{
            "type": "reject",
            "feedback": "Salary increase exceeds budget"
        }]
    }),
    config
)
""")

## Lab 2.4: Complete Approval Workflow Example

In [ ]:
from langgraph.types import Command

print("=" * 70)
print("COMPLETE APPROVAL WORKFLOW")
print("=" * 70)

config2 = {"configurable": {"thread_id": "complete_workflow_1"}}

# Step 1: Initial request
print("\n1️⃣ Employee requests salary update...")
result = hr_approval_agent.invoke(
    {"messages": [{"role": "user", "content": "Update salary for employee 104 to ₹180,000"}]},
    config2
)
print(f"   Status: Execution paused for approval")

# Step 2: Approve with edit
print("\n2️⃣ Manager reviews and edits amount...")
result = hr_approval_agent.invoke(
    Command(resume={
        "decisions": [{
            "type": "edit",
            "args": {"employee_id": "104", "new_salary": 165000}  # Edited amount
        }]
    }),
    config2
)
print(f"   Decision: Approved with edited amount (₹165,000)")
print(f"   Final response: {result['messages'][-1].content}")

print("\n✅ Workflow completed successfully!")

---
# Part 3: Combining Both Middlewares

**Production Pattern:** Use both summarization and approval together

In [ ]:
# Create production-ready agent with BOTH middlewares
production_hr_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance, get_salary_info, update_salary, approve_leave],
    middleware=[
        # Middleware executes in order
        SummarizationMiddleware(
            model="openai:gpt-4o-mini",
            max_tokens_before_summary=4000,
            messages_to_keep=20,
            summary_prompt="Summarize the HR consultation, preserving critical details about employee requests and decisions."
        ),
        HumanInTheLoopMiddleware(
            interrupt_on={
                "update_salary": True,
                "approve_leave": True
            }
        )
    ],
    checkpointer=InMemorySaver(),
    system_prompt="""You are a comprehensive HR assistant.
    
    Capabilities:
    - Employee information and queries
    - Leave management
    - Salary operations (requires approval)
    - Long consultation sessions (with auto-summarization)
    
    You maintain context across long conversations and ensure
    all critical operations get proper approval."""
)

print("✅ Production HR Agent with BOTH middlewares ready!")
print("\nMiddleware Stack:")
print("  1️⃣ SummarizationMiddleware")
print("     - Manages conversation length")
print("     - Preserves recent context")
print("     - Automatic token monitoring")
print("\n  2️⃣ HumanInTheLoopMiddleware")
print("     - Requires approval for sensitive operations")
print("     - Creates workflow interrupts")
print("     - Provides audit trail")
print("\nFeatures:")
print("  📝 Auto-summarization for long conversations")
print("  👤 Human approval for critical operations")
print("  💾 Conversation persistence with checkpointer")
print("  🔒 Security and compliance built-in")
print("  🎯 Production-ready architecture")

## Lab 3.1: Test Combined Middleware in Action

In [ ]:
config = {"configurable": {"thread_id": "combined_demo_1"}}

print("=" * 70)
print("COMBINED MIDDLEWARE DEMONSTRATION")
print("=" * 70)

# Long conversation that will trigger summarization
conversation_steps = [
    "Hi, I'm employee 101. I want to discuss my career.",
    "I've been with the company for 4 years now.",
    "What's my current leave balance?",
    "I'm thinking about taking some time off next month.",
    "Can you tell me about professional development opportunities?",
    "What skills should I focus on to advance my career?",
    "I'd also like to discuss my compensation.",
    "Can you check my current salary?",
    "Please update my salary to ₹150,000."  # This will trigger approval
]

for i, message in enumerate(conversation_steps, 1):
    print(f"\n--- Turn {i} ---")
    print(f"User: {message}")
    
    result = production_hr_agent.invoke(
        {"messages": [{"role": "user", "content": message}]},
        config
    )
    
    print(f"Agent: {result['messages'][-1].content[:150]}...")
    
    # Check for summarization
    if i > 5 and any('summary' in str(m).lower() for m in result.get('messages', [])[:2]):
        print("  📝 [Summarization occurred]")
    
    # Check for approval requirement
    if '__interrupt__' in result:
        print("  ⏸️  [Approval required - execution paused]")
        print(f"  Tool: {result['__interrupt__'][0].value['action_requests'][0]['name']}")
        break

print("\n" + "=" * 70)
print("✅ Demonstration Complete")
print("\nWhat happened:")
print("  1. SummarizationMiddleware managed conversation length")
print("  2. HumanInTheLoopMiddleware intercepted salary update")
print("  3. Agent paused for manager approval")
print("  4. All state preserved in checkpointer")

---
# Summary & Best Practices

## Key Changes in LangChain 1.0

### ✅ Correct Parameters
```python
# Correct
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[...],
    system_prompt="You are a helpful assistant",  # ✅ system_prompt
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"tool_name": True}  # ✅ interrupt_on
        )
    ]
)
```

### ❌ Old/Incorrect Parameters
```python
# Incorrect - will cause errors
agent = create_agent(
    prompt="...",  # ❌ Wrong: use system_prompt
    middleware=[
        HumanInTheLoopMiddleware(
            tool_configs={...}  # ❌ Wrong: use interrupt_on
        )
    ]
)
```

## Built-in Middleware Comparison

| Middleware | Purpose | Key Parameters | When to Use |
|------------|---------|----------------|-------------|
| **SummarizationMiddleware** | Auto-summarize history | `max_tokens_before_summary`, `messages_to_keep`, `summary_prompt` | Long conversations |
| **HumanInTheLoopMiddleware** | Require approval | `interrupt_on` | Critical operations |

## Production Checklist

✅ **SummarizationMiddleware:**
- Set `max_tokens_before_summary` based on your model's context window
- Keep `messages_to_keep` high enough to maintain conversation flow (15-25)
- Customize `summary_prompt` for your domain
- Use a smaller, faster model for summaries (e.g., gpt-4o-mini)

✅ **HumanInTheLoopMiddleware:**
- Configure `interrupt_on` for sensitive operations
- Use `True` for all decisions (approve/edit/reject)
- Use `{"allowed_decisions": ["approve", "reject"]}` to disable editing
- Use `False` for tools that don't need approval
- Always use with a `checkpointer` for state persistence
- Implement approval notification system
- Build UI for approval/rejection
- Store approval audit trail

✅ **Combined Usage:**
- Summarization runs first (manages context)
- Human-in-the-loop runs second (approval gates)
- Both work seamlessly with LangGraph runtime
- State automatically managed by checkpointer

## Approval Workflow Integration

```python
from langgraph.types import Command

# 1. Get interrupt information
result = agent.invoke({"messages": [...]}, config)
if '__interrupt__' in result:
    interrupt = result['__interrupt__'][0]
    action = interrupt.value['action_requests'][0]

# 2. Resume with decision
# Approve
agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}),
    config
)

# Edit
agent.invoke(
    Command(resume={"decisions": [{
        "type": "edit",
        "args": {"new_value": 123}
    }]}),
    config
)

# Reject
agent.invoke(
    Command(resume={"decisions": [{
        "type": "reject",
        "feedback": "Reason for rejection"
    }]}),
    config
)
```

## Migration Guide

1. **Update parameter names:**
   - `prompt` → `system_prompt`
   - `tool_configs` → `interrupt_on`

2. **Import built-in middleware:**
   ```python
   from langchain.agents.middleware import (
       SummarizationMiddleware,
       HumanInTheLoopMiddleware
   )
   ```

3. **Update agent creation:**
   ```python
   agent = create_agent(
       model="openai:gpt-4o-mini",
       tools=[...],
       system_prompt="Your instructions",
       middleware=[
           SummarizationMiddleware(...),
           HumanInTheLoopMiddleware(interrupt_on={...})
       ],
       checkpointer=InMemorySaver()
   )
   ```

## Resources

- [LangChain Middleware Documentation](https://docs.langchain.com/oss/python/langchain/middleware)
- [Human-in-the-Loop Documentation](https://docs.langchain.com/oss/python/langchain/human-in-the-loop)
- [LangGraph Checkpointers](https://langchain-ai.github.io/langgraph/concepts/persistence/)
- [Agent Middleware Blog Post](https://blog.langchain.com/agent-middleware/)

---

**Congratulations!** You now know how to use LangChain 1.0's built-in class-based middleware correctly! 🎉

**Key Takeaways:**
- ✅ Use `system_prompt` not `prompt`
- ✅ Use `interrupt_on` not `tool_configs`
- ✅ Built-in, production-tested middleware
- ✅ Seamless LangGraph integration
- ✅ Clean, maintainable code
- ✅ Ready for enterprise deployment